In [ ]:
#Import
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *
import threading
import time
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timezone
from datetime import timedelta

#Class for Interactive Brokers Connection
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
    #Listen for realtime bars
    def updateMktDepth(self, reqId, position: int, operation: int, side: int, price: float, size):
        bot.market_depth_update(reqId, position, operation, side, price, size)

#Bot Logic
class Bot():
    ib = None
    
    def __init__(self, Minute):
        self.Minute = int(Minute)
        self.insert_ask_list = []
        self.insert_bid_list = []
        self.bid0_list = []
        self.ask0_list = []
        self.insert_ask_vol_list = []
        self.insert_bid_vol_list = []
        self.bid0_vol_list = []
        self.ask0_vol_list = []
        
        #Connect to IB on init
        self.ib = IBapi()
        self.ib.connect ('127.0.0.1', 7496, 1)
        ib_thread = threading.Thread(target=self.run_loop, daemon=True)
        ib_thread.start()
        time.sleep(1)
        
        #Create our IB contract object
        contract = Contract()
        contract.symbol = "EUR"
        contract.secType ="CASH"
        contract.exchange = "IDEALPRO"
        contract.currency = "USD"
        
        # Request Market Data
        self.ib.reqMktDepth(0, contract, 20, False, [])
    
    #Listen to socket in seperate thread
    def run_loop(self):
        self.ib.run()

    #Pass realtime bar data back to our bot object
    def market_depth_update(self, reqId, position, operation, side, price, size):
        #operation type
        if operation==0 :
            operation='insert'
        elif operation==1 :
            operation='update'
        else:
            operation='delete'
        
        price=str(price)
        size=str(size)
        
        #oder type
        if side==0:
            side='ask'
        else:
            side='bid'

        DateTime = datetime.now(timezone(timedelta(hours=+2)))
        minute = int(DateTime.strftime("%M"))
        
        
        #print(DateTime.strftime("%m-%d-%Y, %H:%M:%S.%f"))
        if len(price[4:])>=3 and (operation == 'insert' or position==0):
            
            if operation == 'insert' and side=='ask':
                if int(price[4:]) in self.insert_ask_list:
                    self.insert_ask_vol_list[self.insert_ask_list.index(int(price[4:]))] += int(size)/100000
                else:
                    self.insert_ask_list.append(int(price[4:]))
                    self.insert_ask_vol_list.append(int(size)/100000)
            if operation == 'insert' and side=='bid':
                if int(price[4:]) in self.insert_bid_list:
                    self.insert_bid_vol_list[self.insert_bid_list.index(int(price[4:]))] += int(size)/100000
                else:
                    self.insert_bid_list.append(int(price[4:]))
                    self.insert_bid_vol_list.append(int(size)/100000)
            elif side=='bid':
                if int(price[4:]) in self.bid0_list:
                    self.bid0_vol_list[self.bid0_list.index(int(price[4:]))] += int(size)/100000
                else:
                    self.bid0_list.append(int(price[4:]))
                    self.bid0_vol_list.append(int(size)/100000)
            else:
                if int(price[4:]) in self.ask0_list:
                    self.ask0_vol_list[self.ask0_list.index(int(price[4:]))] += int(size)/100000
                else:
                    self.ask0_list.append(int(price[4:]))
                    self.ask0_vol_list.append(int(size)/100000)
            
            if minute != self.Minute:
                print("minute =", minute)
                print("Minute", self.Minute)
                plt.figure()
                plt.style.use('ggplot')
                plt.bar(self.insert_bid_list, self.insert_bid_vol_list, alpha=0.5, label='insert bid', color='orange')
                plt.bar(self.insert_ask_list, self.insert_ask_vol_list, alpha=0.5, label='insert ask', color='blue')
                plt.bar(self.bid0_list, self.bid0_vol_list, alpha=0.5, label='bid', color='red')
                plt.bar(self.ask0_list, self.ask0_vol_list, alpha=0.5, label='ask', color='green')
                plt.legend(loc='upper right')
                plt.xlabel('price')
                plt.ylabel('volume')
                plt.title('insert & bid orders')
                plt.show()
                
                self.Minute = minute
                self.insert_bid_list, self.insert_bid_vol_list = [],[]
                self.insert_ask_list, self.insert_ask_vol_list = [],[]
                self.bid0_list, self.bid0_vol_list = [],[]
                self.bid0_list, self.bid0_vol_list = [],[]
                
            
        print(f'POSITION : {position} OPERATION : {operation} SIDE : {side} PRICE : {price}$  SIZE :{size}')
            
            
            
            
#Start bot

hour = int(datetime.now(timezone(timedelta(hours=+2))).strftime('%H'))
Minute = int(datetime.now(timezone(timedelta(hours=+2))).strftime('%M'))
day = datetime.now(timezone(timedelta(hours=+2))).strftime('%a')

if day != 'Sat' and day != 'Sun' and day != 'Fri':
    bot = Bot(Minute)
elif day=='Fri' and hour<22:
    bot = Bot(Minute)
elif day=='Fri' and hour==22 and Minute<55:
    bot = Bot(Minute)